In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import numpy as np
import os.path

path = 'data/'
if not os.path.isfile('001_Data_retrieve.ipynb'):
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/TRABAJO/Data Science/ITBA-DeepLearning/Notebooks/TP-FINAL/bioinformatics_final_project/data/'
    !pip install scikit-optimize

from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Integer, Categorical
from skopt.plots import plot_convergence
from skopt.utils import use_named_args
from skopt import dump, load
from skopt import BayesSearchCV
from skopt.callbacks import CheckpointSaver
from skopt import load
from skopt import callbacks
from skopt.callbacks import CheckpointSaver
from skopt.callbacks import DeadlineStopper # Stop the optimization before running out of a fixed budget of time.
from skopt.callbacks import VerboseCallback # Callback to control the verbosity
from skopt.callbacks import DeltaXStopper # Stop the optimization If the last two positions at which the objective has been evaluated are less than delta
from skopt.plots import plot_evaluations
from skopt.plots import plot_objective
from skopt.plots import plot_convergence
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
    

Mounted at /content/drive
     |████████████████████████████████| 101 kB 5.4 MB/s 


## Carga de Datos

In [ ]:
df_completo = pd.read_csv(path+'acetylcholinesterase_02_bioactivity_data_preprocessed_token_descriptors.csv')
X= df_completo.drop(['molecule_chembl_id', 'canonical_smiles', 'standard_value',
       'standard_value_norm', 'pIC50', 'X_seq', 'X_seq_pad', 'MW', 'LogP',
       'NumHDonors', 'NumHAcceptors', 'bioactivity_class', 'Name'], axis=1)
y = df_completo.pIC50.values

## Split de Datos

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

## Modelo LighGBM

In [ ]:
def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
def R2_numpy(y_true, y_pred):
    SS_res =  np.sum(np.square( y_true-y_pred ))
    SS_tot = np.sum(np.square( y_true - np.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + np.finfo(float).eps ) )

In [ ]:
cat_vars = X.columns.to_list()
min_child_samples=100  # cant minima de hoja hija para hacer split
n_estimators=800000  # cant max de arboles secuenciales, por lo general se pone numero alto, ya q nunca llega por early stopping
learning_rate=0.005
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'r2',
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars  # le paso cuales son los cat para q haga EFB(exlusive Feature Bunding)
           }

In [ ]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PubchemFP0', 'PubchemFP1', 'PubchemFP10', 'PubchemFP100', 'PubchemFP101', 'PubchemFP102', 'PubchemFP103', 'PubchemFP104', 'PubchemFP105', 'PubchemFP106', 'PubchemFP107', 'PubchemFP108', 'PubchemFP109', 'PubchemFP11', 'PubchemFP110', 'PubchemFP111', 'PubchemFP112', 'PubchemFP113', 'PubchemFP114', 'PubchemFP115', 'PubchemFP116', 'PubchemFP117', 'PubchemFP118', 'PubchemFP119', 'PubchemFP12', 'PubchemFP120', 'PubchemFP121', 'PubchemFP122', 'PubchemFP123', 'PubchemFP124', 'PubchemFP125', 'PubchemFP126', 'PubchemFP127', 'PubchemFP128', 'PubchemFP129', 'PubchemFP13', 'PubchemFP130', 'PubchemFP131', 'PubchemFP132', 'PubchemFP133', 'PubchemFP134', 'PubchemFP135', 'PubchemFP136', 'PubchemFP137', 'PubchemFP138', 'PubchemFP139', 'PubchemFP14', 'PubchemFP140', 'PubchemFP141', 'PubchemFP142', 'PubchemFP143', 'PubchemFP144', 'PubchemFP145', 'PubchemFP1

Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 2.32577
[200]	valid's l2: 2.16832
[300]	valid's l2: 2.07787
[400]	valid's l2: 2.02185
[500]	valid's l2: 1.97655
[600]	valid's l2: 1.94276
[700]	valid's l2: 1.92298
[800]	valid's l2: 1.89901
[900]	valid's l2: 1.87331
[1000]	valid's l2: 1.85773
[1100]	valid's l2: 1.84224
[1200]	valid's l2: 1.82627
[1300]	valid's l2: 1.8114
[1400]	valid's l2: 1.80057
[1500]	valid's l2: 1.7905
[1600]	valid's l2: 1.78198
[1700]	valid's l2: 1.77215
[1800]	valid's l2: 1.76306
[1900]	valid's l2: 1.75799
[2000]	valid's l2: 1.75047
[2100]	valid's l2: 1.74617
[2200]	valid's l2: 1.74258
[2300]	valid's l2: 1.73904
[2400]	valid's l2: 1.73606
[2500]	valid's l2: 1.73355
[2600]	valid's l2: 1.72939
[2700]	valid's l2: 1.72404
[2800]	valid's l2: 1.72139
[2900]	valid's l2: 1.71945
[3000]	valid's l2: 1.7178
[3100]	valid's l2: 1.71484
[3200]	valid's l2: 1.71188
[3300]	valid's l2: 1.71165
Early stopping, best iteration is:
[3224]	valid's l2: 1.71

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.005, max_depth=-1,
              min_child_samples=100, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=800000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
model.score(X_val, y_val)

0.35426855506091404

In [ ]:
y_val_pred = model.predict(X_val)
print('val_R2: ',R2_numpy(y_val, y_val_pred))

val_R2:  0.35426855506091404


## HYPER-SEARCH PARAMETERS

In [ ]:
#Defino con que hyper-parametros realizo la busqueda
dim_learning_rate= Real(low=0.01, high=1, prior='log-uniform', name='learning_rate')
dim_boosting_type = Categorical(['gbdt'], name='boosting_type')
dim_subsample = Real(low=0.01, high=1.0, prior='log-uniform', name='subsample')
dim_subsample_freq = Integer(0, 10, name='subsample_freq')
dim_max_depth= Integer(1, 20, name='max_depth') # 	Larger is usually better, but overfitting speed increases.
dim_num_leaves= Integer(2, 100,name='num_leaves') #max number of leaves in one tree        
dim_min_child_samples= Integer(1, 200, name='min_child_samples') # minimal number of data in one leaf
dim_reg_lambda= Real(0.001, 100, 'log-uniform', name='reg_lambda') # L2 regularization
dim_reg_alpha= Real(0.001, 100, 'log-uniform', name='reg_alpha') # L1 regularization
dim_colsample_bytree= Real(0.1, 1.0, 'uniform', name='colsample_bytree') # enabler of bagging fraction
dim_min_child_weight=Integer(0, 10, name='min_child_weight') # minimal number of data in one leaf.
dim_n_estimators= Integer(1, 1000, name='n_estimators') # cant. de estimadores secuenciales (se pone alto stopea earlystopping)        
dimensions = [
    #Regularizacion --> complejidad del modelo
    dim_max_depth,
    dim_min_child_weight,
    dim_reg_lambda,
    dim_reg_alpha,

    #Regularizacion --> aletoriedad
    dim_subsample, 
    dim_subsample_freq,
    dim_colsample_bytree,

    #Otros
    dim_num_leaves,
    dim_min_child_samples,            
    dim_n_estimators,
    dim_boosting_type,
    

]


In [ ]:
max_iterSearch = 300
best_accuracy = -10.0
best_parameters = [{'teste': 1}]

# Callback de Checkpoint Saver 
if not os.path.isfile('001_Data_retrieve.ipynb'):
    checkpoint_saver = CheckpointSaver(path+'/checkpoint.pkl', compress=9)
else:
    checkpoint_saver = CheckpointSaver('checkpoint.pkl', compress=9)
#deltaXStopper = DeltaXStopper(0.001) 
#deltaLineStoper = DeadlineStopper(60*5)


# Funcion para LightGBM y skopt
@use_named_args(dimensions=dimensions)
def fitness (max_depth, num_leaves, min_child_samples,reg_lambda, reg_alpha, colsample_bytree, min_child_weight,
            boosting_type, subsample, subsample_freq, n_estimators):
   

    model = LGBMRegressor( max_depth=max_depth,
                          min_child_samples=min_child_samples, 
                          reg_lambda=reg_lambda,
                          subsample=subsample, subsample_freq=subsample_freq,
                          colsample_bytree=colsample_bytree,

                          #num_leaves=num_leaves, 
                          #reg_alpha=reg_alpha,
                          #min_child_weight= min_child_weight, n_estimators=n_estimators,
                          #boosting_type=boosting_type, 
                         n_jobs=2)
    
    ### PARA CON CV ###
    #fit_params={"early_stopping_rounds":30, 
    #      "eval_metric" : 'r2', 
    #      #"eval_set" : [(X_val, y_val.reshape(-1))],
    #      #'eval_names': ['valid'],
    #      'verbose': 0,
    #      'feature_name': 'auto', # that's actually the default
    #      #'categorical_feature': cat_vars
    #   }
    #score = np.mean(cross_val_score(model, X_train, y_train.reshape(-1), cv=KFold(n_splits=4), n_jobs=2, verbose=0
    #                ,scoring= 'r2'
    #                , fit_params= fit_params))
    ###-------------

    ### PARA SIN CV ##
    fit = model.fit(X_train, y_train.reshape(-1), 
                eval_set=[(X_val, y_val.reshape(-1))],
                eval_metric='r2', 
                #feature_name='auto', 
                #categorical_feature=cat_vars, 
                verbose=False,
                early_stopping_rounds=100)
    y_pred  =  model.predict(X_val)  
    score = R2_numpy(y_val, y_pred)
    ###----------------
    

    global best_accuracy
    global best_parameters
    
    if score > best_accuracy:
        #print('mejor score actual:', score)
        #print('mejor score anterior:', best_accuracy)
        best_parameters[0] = model.get_params()        
        best_accuracy = score
        
    del model
    
    return -score


In [ ]:
%time search_result = gp_minimize(func=fitness, dimensions=dimensions, n_calls=max_iterSearch, n_jobs=2, verbose=True, acq_func='LCB',callback=[checkpoint_saver])

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.2850
Function value obtained: -0.0962
Current minimum: -0.0962
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2196
Function value obtained: -0.1399
Current minimum: -0.1399
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2096
Function value obtained: -0.0978
Current minimum: -0.1399
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1788
Function value obtained: 0.0007
Current minimum: -0.1399
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1847
Function value obtained: 0.0007
Current minimum: -0.1399
Iteration No: 6 started. Ev

In [ ]:
#PRINT DE RESULTADO

print('best score custom function', best_accuracy)
#print('best score gp_minimize', search_result.fun)
print('best parametrers gp_minimize')
print('best parametrers custom function:')
best_parameters

best score custom function 0.3664414680681032
best parametrers gp_minimize
best parametrers custom function:


[{'boosting_type': 'gbdt',
  'class_weight': None,
  'colsample_bytree': 0.6205746891155753,
  'importance_type': 'split',
  'learning_rate': 0.1,
  'max_depth': 20,
  'min_child_samples': 50,
  'min_child_weight': 0.001,
  'min_split_gain': 0.0,
  'n_estimators': 100,
  'n_jobs': 2,
  'num_leaves': 31,
  'objective': None,
  'random_state': None,
  'reg_alpha': 0.0,
  'reg_lambda': 0.001,
  'silent': True,
  'subsample': 0.7840301786541193,
  'subsample_for_bin': 200000,
  'subsample_freq': 0}]

In [ ]:
#plot_convergence(search_result)
#ppE = plot_evaluations(search_result)
#pp = plot_objective(search_result)

In [ ]:
model = LGBMRegressor(**best_parameters[0])

model.fit(X_train, y_train.reshape(-1), **fit_params)
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['PubchemFP0', 'PubchemFP1', 'PubchemFP10', 'PubchemFP100', 'PubchemFP101', 'PubchemFP102', 'PubchemFP103', 'PubchemFP104', 'PubchemFP105', 'PubchemFP106', 'PubchemFP107', 'PubchemFP108', 'PubchemFP109', 'PubchemFP11', 'PubchemFP110', 'PubchemFP111', 'PubchemFP112', 'PubchemFP113', 'PubchemFP114', 'PubchemFP115', 'PubchemFP116', 'PubchemFP117', 'PubchemFP118', 'PubchemFP119', 'PubchemFP12', 'PubchemFP120', 'PubchemFP121', 'PubchemFP122', 'PubchemFP123', 'PubchemFP124', 'PubchemFP125', 'PubchemFP126', 'PubchemFP127', 'PubchemFP128', 'PubchemFP129', 'PubchemFP13', 'PubchemFP130', 'PubchemFP131', 'PubchemFP132', 'PubchemFP133', 'PubchemFP134', 'PubchemFP135', 'PubchemFP136', 'PubchemFP137', 'PubchemFP138', 'PubchemFP139', 'PubchemFP14', 'PubchemFP140', 'PubchemFP141', 'PubchemFP142', 'PubchemFP143', 'PubchemFP144', 'PubchemFP145', 'PubchemFP1

Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 1.70348
Did not meet early stopping. Best iteration is:
[89]	valid's l2: 1.69999


In [ ]:
print('R2 para train:', R2_numpy(y_train, y_pred_train))
print('R2 para val  :', R2_numpy(y_val, y_pred_val))

R2 para train: 0.542245037365517
R2 para val  : 0.358296520632047
